In [1]:
init_jupyter()
import krpc

Got exception No module named 'unicorn'
Runnign magic qt5
done
False


In [2]:
conn = krpc.connect(
    name='My Example Program',
    address='localhost',
    rpc_port=50000, stream_port=50001)
print(conn.krpc.get_status().version)

0.4.9


In [5]:
conn.nop(req_phys_loop=True)

In [3]:

conn.space_center.launch_vessel2('VAB', "./Ships/VAB/SpaceX Falcon 9 Block 5.craft", 'LaunchPad', True)

In [2]:
import krpc.schema.KRPC_pb2 as KRPC
x = KRPC.Request()
x.lock_update = False
conn.nop()


NameError: name 'conn' is not defined

In [4]:
cam = conn.space_center.camera

In [9]:
import time
time.sleep(1)
cam.pitch = 30

In [23]:
import time
for i in range(100):
    conn.lock_update = False
    conn.wait_req_id = i+1
    conn.req_id = i
    conn.nop()
    time.sleep(0.5)
    conn.wait_req_id = i+1
    conn.req_id = i+1
    conn.nop()
    time.sleep(0.5)
#vessel = conn.space_center.active_vessel
#flight_info = vessel.flight()

In [39]:
conn.space_center.physics_warp_factor = 1
conn.space_center.time_warp_helper.set_physical_warp_rate(0, 0.3)
conn.space_center.time_warp_helper.set_warp_rate(0, 1)
conn.space_center.physics_warp_factor = 0

In [10]:
vessel = conn.space_center.active_vessel
flight_info = vessel.flight()
earth = conn.space_center.bodies['Kerbin']
flight = vessel.flight(earth.reference_frame)

In [4]:
a=  earth.reference_frame

In [5]:
a.direction_to_world_space([1,1,0])

(-0.766054489496099, 1.0, -0.6427756366904962)

In [69]:
alt=earth.altitude_at_position(flight.center_of_mass, earth.reference_frame)
lat=earth.latitude_at_position(flight.center_of_mass, earth.reference_frame)
lon=earth.longitude_at_position(flight.center_of_mass, earth.reference_frame)
alt,lat,lon

(102.6105435580248, -0.09722078534075375, -74.55779311997983)

In [ ]:
flight.world_relative_velocity(earth, flight.center_of_mass, flight.velocity)

In [8]:
flight.simulate_aerodynamic_force_at(earth, flight.center_of_mass, flight.velocity)

(-0.056606455597542724, 0.045636966824531555, -0.03355812907990103)

In [19]:
np.linalg.norm(conn.space_center.aerodynamics.sim_aero_force_by_alt(earth, vessel, [0,100,0], 1000))

721422.8663024207

In [58]:
px = earth.surface_position(lat, lon, earth.reference_frame)
earth.pos_to_body_velocity(px, earth.reference_frame)

(-109.99548720577155, -0.0, -136.06315710207622)

In [66]:
flight.velocity

(0.0011362587037240246, 0.0012776755589566874, -0.0006310238968652016)

In [55]:
vessel.bounding_box(earth.reference_frame)

((159775.21875, -1021.536376953125, -578477.375),
 (159800.28125, -1015.2615966796875, -578406.0))

In [81]:
flight.simulate_aerodynamic_force_at(earth,flight.center_of_mass,flight.velocity)

(-0.0040933823838829675, 0.04019081965088844, -0.0005925529566284283)

In [ ]:
earth.reference_frame.

In [42]:
vessel.bounding_box(vessel.reference_frame)
vessel.inertia_tensor

((-2.6765403747558594, -30.703201293945312, -2.6734657287597656),
 (2.6756887435913086, 38.58135986328125, 2.679896831512451))

In [27]:
for x in vessel.active_engines:
    print(x.available_torque)

((690854.248046875, 0.0, 690851.5625), (-690854.248046875, -0.0, -690851.5625))
((842755.4931640625, 57895.2751159668, 842754.7607421875), (-842755.4931640625, -57895.2751159668, -842754.7607421875))
((842857.4829101562, 57965.99197387695, 842854.8583984375), (-842857.4829101562, -57965.99197387695, -842854.8583984375))
((940255.6762695312, 46290.45867919922, 940255.5541992188), (-940255.6762695312, -46290.45867919922, -940255.5541992188))
((842892.6391601562, 57823.04000854492, 842889.892578125), (-842892.6391601562, -57823.04000854492, -842889.892578125))
((940153.4423828125, 46474.33090209961, 940153.564453125), (-940153.4423828125, -46474.33090209961, -940153.564453125))
((842767.822265625, 57961.936950683594, 842765.625), (-842767.822265625, -57961.936950683594, -842765.625))
((940208.740234375, 46335.0944519043, 940207.8857421875), (-940208.740234375, -46335.0944519043, -940207.8857421875))
((842807.4951171875, 58094.40231323242, 842805.17578125), (-842807.4951171875, -58094.4023

In [141]:
rscs = set()
for engine in vessel.active_engines:
    for prop in engine.propellants:
        for rsc in vessel.resources.with_resource_by_id(prop.id):
            rscs.add(rsc)
import time
def set_prop(v, target_name=None):
    for rsc in rscs:
        if target_name is None or target_name == rsc.part.name:
            rsc.amount = rsc.max * v
xl = np.linspace(0, 1, num=2)

In [190]:
tensors = []
masses = []
poslist = []
for lvl in xl:
    set_prop(lvl)
    time.sleep(0.1)
    tensors.append(vessel.inertia_tensor)
    masses.append(vessel.mass)
    poslist.append(vessel.position(earth.reference_frame))
    
    for engine in vessel.active_engines:
        print(engine.thrusters, engine.max_thrust, engine.gimballed)
        for th in engine.thrusters:
            if th.gimballed:
                print(th.thrust_position(vessel.reference_frame))
                print(th.thrust_direction(vessel.reference_frame))
            pass
        break

[<SpaceCenter.Thruster remote object #690>] 376748.09375 True
(0.10783513186313343, -40.87232427900392, 0.10682853920706553)
(-0.07006975104751101, 0.995121921654327, -0.06944741113410989)
[<SpaceCenter.Thruster remote object #690>] 376747.625 True
(0.1092348610096856, -27.24575482834026, 0.1003312087343944)
(-0.07092597235582093, 0.9952683594849521, -0.06641348529711708)


In [188]:
poslist

[(159789.77336466333, -1018.162910724408, -578449.2078686076),
 (159786.18044517605, -1018.2516256965464, -578435.9886531583)]

In [104]:
oplt.plot(K.Dataset(tx[:,0]))

QAPP
CLOSING  <PyQt5.QtGui.QCloseEvent object at 0x7fe26f99f010>


In [126]:
tx = np.array(tensors)
p = np.poly1d(np.polyfit(xl, tx[:,0], 3))
px = oplt.plot(K.Dataset(tx[:,0]), o=1)

QAPP


In [127]:
px.w.add_plot(K.Dataset(p(xl)))
px.w.add_plot(K.Dataset(tx[:,0]))

In [134]:
e0 = vessel.active_engines[0]
e0.gimbal_range
e0.available_torque


((0.0, 0.0, 0.0), (-0.0, -0.0, -0.0))

In [138]:
t0 = e0.thrusters[0]
t0.gimballed

False

In [144]:
e0.thrusters[0].gimballed
e0.gimballed

True

In [178]:
vessel.control.pitch = -1
vessel.control.yaw =1 

In [179]:
for engine in vessel.active_engines:
    print(engine.thrusters, engine.max_thrust, engine.gimballed)
    for th in engine.thrusters:
        if th.gimballed:
            print(th.thrust_position(vessel.reference_frame))
            print(th.thrust_direction(vessel.reference_frame))
        pass

[<SpaceCenter.Thruster remote object #690>] 376747.5 True
(0.10886742455599219, -27.700780248834228, 0.10041771496390575)
(-0.07073812361546022, 0.9952783905444449, -0.0664609706492645)
[<SpaceCenter.Thruster remote object #692>] 376748.96875 True
(-1.0927694089494935, -27.566714770134205, 0.5777311493726317)
(-0.06572933887892546, 0.9963702456803455, -0.054091026873301286)
[<SpaceCenter.Thruster remote object #694>] 376748.9375 True
(-0.4076294337257913, -27.56870934137718, 1.2954090368686022)
(-0.05645445392993681, 0.9962028296718544, -0.06627920130225377)
[<SpaceCenter.Thruster remote object #696>] 376748.90625 True
(0.5835151374710339, -27.570484178967007, 1.2616356120844727)
(-0.058002663178993163, 0.9973314259831418, -0.04434256951203752)
[<SpaceCenter.Thruster remote object #698>] 376748.90625 True
(1.280005781163256, -27.56486351842969, 0.5957898806803411)
(-0.05612785192336217, 0.9962483037782478, -0.06587763216114804)
[<SpaceCenter.Thruster remote object #700>] 376748.84375 T

In [120]:
for engine in vessel.active_engines:
    for prop in engine.propellants:
        print()
        print(prop.name, prop.total_resource_available, prop.current_amount)
        for rsc in vessel.resources.with_resource_by_id(prop.id):
            rsc.amount = rsc.max


LiquidFuel 8457.113153636616 1.8282749140304368e-05

Oxidizer 10336.471280363887 2.2345583358877484e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.234558335887748e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.234558335887748e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.234558335887748e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.2345583358877484e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.2345583358877484e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.2345583358877484e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.2345583358877484e-05

LiquidFuel 14434.2001953125 1.8282749140304368e-05

Oxidizer 17641.80078125 2.2345583358877484e-05


In [53]:
for engine in vessel.active_engines:
    for prop in engine.propellants:
        print()
        print(prop.name, prop.total_resource_available, prop.current_amount)
        for rsc in vessel.resources.with_resource_by_id(prop.id):
            print(rsc.amount)


LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.32270469536201674
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.2001953125

Oxidizer 0.0 0.36670563121721145
2640.0
0.0

LiquidFuel 14434.2001953125 0.3000318656356633
2160.0
14434.200

In [29]:
for engine in vessel.active_engines:
    rsc = vessel.resources.with_resource_by_id(x.propellants[0].id)
    print(rsc)
    

[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]
[<SpaceCenter.Resource remote object #108>, <SpaceCenter.Resource remote object #115>]


In [10]:
vessel.inertia_tensor

[1396.365234375,
 0.0,
 4.993131005903706e-05,
 0.0,
 1648.2491455078125,
 0.0,
 4.4811466068495065e-05,
 0.0,
 1396.3648681640625]

In [113]:
vessel = conn.space_center.active_vessel
flight_info = vessel.flight()
earth = conn.space_center.bodies['Kerbin']
flight = vessel.flight(earth.reference_frame)

In [7]:
cheats = conn.space_center.cheats
cheats.unbreakable_joints=True
cheats.no_crash_damage=True
cheats.ignore_max_temperature=True

In [11]:
vessel.control.sas = False
vessel.control.rcs = False
vessel.control.throttle = 1.0
vessel.control.activate_next_stage()
vessel.control.activate_next_stage()

[<SpaceCenter.Vessel remote object #28>,
 <SpaceCenter.Vessel remote object #29>,
 <SpaceCenter.Vessel remote object #30>,
 <SpaceCenter.Vessel remote object #31>]

In [33]:
vessel.control.throttle=1

In [119]:
vpos = vessel.position(earth.reference_frame)
flight.set_position(tuple(np.array(vpos)*1.5))
conn.nop(req_phys_loop=1)
flight.set_position(tuple(np.array(vpos)*1.9))
conn.nop(req_phys_loop=1)
flight.set_position(tuple(np.array(vpos)*3.9))
conn.nop(req_phys_loop=1)

In [66]:
cam = conn.space_center.camera
cam.mode = conn.space_center.CameraMode.locked
cam.yaw = 90
cam.pitch = 0

In [32]:
set_prop(1)

In [156]:
tank = [x for x in vessel.parts.all if 'S1tank' in x.name ][0]
t0 = [x for x in vessel.parts.all if 'Merlin' in x.name ][0]

In [105]:
set_prop(0)
conn.nop(req_phys_loop=1)
print(t0.center_of_mass(vessel.reference_frame), tank.mass-tankmass)
set_prop(0.5)
conn.nop(req_phys_loop=1)
print(t0.center_of_mass(vessel.reference_frame), tank.mass-tankmass)
set_prop(1)
conn.nop(req_phys_loop=1)
print(t0.center_of_mass(vessel.reference_frame), tank.mass-tankmass, v0_mass)


(8.696231615701322e-06, 10.992742672733808, -5.018335593298673e-06) 0.0
(6.255828032486477e-06, 22.755642570073487, -3.1816487648939074e-06) 80189.9921875
(1.3838243845043507e-05, 24.946099593443158, -8.694243124551804e-06) 160380.0 42191.5


In [140]:
vessel.active_engines[0].part.name

'KK.SPX.Merlin1D++'

In [144]:
tank.massless

False

In [164]:
np.linalg.norm(t0.center_of_mass(tank.reference_frame))

23.567829728436717

In [166]:
tank.name

'KK.SPX.F93.S1tank'

In [167]:
d0

4.152799920786924

In [162]:
def an1(x):
    set_prop(x, target_name=tank.name)
    conn.nop(req_phys_loop=1)
    m1 = tank.mass -tankmass
    print(m1)
    dist = np.linalg.norm(t0.center_of_mass(vessel.reference_frame))-d0
    print(t0.center_of_mass(vessel.reference_frame))
    return dist * (m1 + v0_mass) / m1
[an1(x) for x in np.linspace(0.001, 1, 10)] 

160.3798828125
(0.006607840066739645, 4.2155447102393095, 0.009847815930339232)
17962.55859375
(0.007108003894695125, 9.506948744708545, 0.011525101003263316)
35764.73828125
(0.0064112655247750006, 12.772777331293991, 0.012824767878204746)
53566.91796875
(0.006244230760803937, 14.989191526731208, 0.01356100375587932)
71369.1015625
(0.006051635256660859, 16.591982404543536, 0.014102855139617176)
89171.28125
(0.00585654157048765, 17.80499704569123, 0.014539374408505168)
106973.453125
(0.005692010959387428, 18.75500009044441, 0.014889232279530873)
124775.640625
(0.00555333472032693, 19.51916674943078, 0.015173764508171317)
142577.828125
(0.00543431300344821, 20.147164640145906, 0.01540880600973793)
160380.0
(0.005329767139030217, 20.672417723015343, 0.015608916553462748)


[22.459951078394624,
 22.41393461292237,
 22.414352799244583,
 22.4145347268307,
 22.414631912346785,
 22.414704776697008,
 22.414756568593756,
 22.414795261727292,
 22.414821595569848,
 22.41484773961267]

In [157]:

set_prop(0)
conn.nop(req_phys_loop=1)
print(tank.center_of_mass(vessel.reference_frame))
set_prop(1)
conn.nop(req_phys_loop=1)
print(tank.center_of_mass(vessel.reference_frame))

(0.002997979814360896, -15.2356113496811, -0.004315002600176587)
(0.0005079184067136033, -5.8953978630925175, -0.0021187432084524005)


In [169]:
set_prop(1)
conn.nop(req_phys_loop=1)
mass_prop = tank.mass-tankmass

set_prop(0, target_name=tank.name)
conn.nop(req_phys_loop=1)
d0= np.linalg.norm(t0.center_of_mass(vessel.reference_frame))
tankmass = tank.mass
v0_mass = vessel.mass
print(np.linalg.norm(tank.center_of_mass(vessel.reference_frame)))

22.415024489670547


In [160]:
for i, p in enumerate(vessel.parts.all):
    set_prop(1, target_name=tank.name)
    conn.nop(req_phys_loop=1)
    print(i, p.name, p.mass)
    set_prop(0, target_name=tank.name)
    conn.nop(req_phys_loop=1)
    print(i, p.name, p.mass)

0 KK.SPX.F9.1875mmPA 150.0
0 KK.SPX.F9.1875mmPA 150.0
1 KK.SPX.F93.S2tank 40400.0
1 KK.SPX.F93.S2tank 40400.0
2 KK.SPX.Merlin1DV+ 1500.0
2 KK.SPX.Merlin1DV+ 1500.0
3 KK.SpX.Mvac.skirt 14.999999046325684
3 KK.SpX.Mvac.skirt 14.999999046325684
4 KK.SPX.Falcon9.FC 150.0
4 KK.SPX.Falcon9.FC 150.0
5 KK.SPX.F93.interstage 590.0
5 KK.SPX.F93.interstage 590.0
6 KK.SPX.FalconGridfin.FT 47.5
6 KK.SPX.FalconGridfin.FT 47.5
7 KK.SPX.FalconGridfin.FT 47.5
7 KK.SPX.FalconGridfin.FT 47.5
8 KK.SPX.FalconGridfin.FT 47.5
8 KK.SPX.FalconGridfin.FT 47.5
9 KK.SPX.F93.S1tank 169006.0
9 KK.SPX.F93.S1tank 8626.0
10 KK.SPX.Falcon9.FC 150.0
10 KK.SPX.Falcon9.FC 150.0
11 KK.SPX.F9LandingLeg 150.0
11 KK.SPX.F9LandingLeg 150.0
12 KK.SPX.F9LandingLeg 150.0
12 KK.SPX.F9LandingLeg 150.0
13 KK.SPX.F9LandingLeg 150.0
13 KK.SPX.F9LandingLeg 150.0
14 KK.SPX.F9LandingLeg 150.0
14 KK.SPX.F9LandingLeg 150.0
15 KK.SPX.F9FT.Interstage.RCS 0.0
15 KK.SPX.F9FT.Interstage.RCS 0.0
16 KK.SPX.F9FT.Interstage.RCS 0.0
16 KK.SPX.F9FT.I

In [ ]:
e0.

In [58]:
cam.transform.rotation = [0,0,0,1]

In [55]:
from scipy.spatial.transform import Rotation as R
R.identity().as_quat()

array([0.000, 0.000, 0.000, 1.000])

In [62]:
cam.pitch, cam.yaw

(2.3035025596618652, -0.3382159173488617)

In [57]:
e0  = vessel.active_engines[0]

In [ ]:
conn.space_center

In [40]:
import time
e0  = vessel.active_engines[0]
e0.gimbal.enable_gimbal = False
for i in np.linspace(-1, 1, 2):
    for j in np.linspace(-1, 1, 5):
        time.sleep(1)
        e0.gimbal.set_gimbal_rot(e0.gimbal.actuation2_rot([i,j]))

In [ ]:

e0.gimbal.set_gimbal_rot(e0.gimbal.actuation2_rot([i,j]))

In [61]:
cam.set_target_vessel(vessel)

In [63]:
cam.distance = 100

In [49]:
cam.max_pitch

88.8084487915039

In [50]:
cam.pitch, cam.yaw

(-0.4010736346244812, -179.7942352294922)

In [59]:
cam.set_target_part(e0.part)

In [33]:
cam.pitch

-57.570491790771484

In [391]:
flight.set_rotation(tuple(unity_ylookat([1,0,0]).as_quat()))

In [ ]:
unity_ylookat([1,0,0]).as_quat()

In [114]:
for i in range(10):
    flight.set_rotation(tuple(unity_ylookat([1,0,0]).as_quat()))
    time.sleep(1)

In [397]:
flight.aerodynamic_force

(0.5600353257189661, 0.4150381992076291, -0.05589944213070355)

In [450]:
from scipy.spatial.transform import Rotation as R
import numpy as np
import time
def unity_ylookat(dy ,dz=None):
    dy = np.array(dy)
    if dz is None: dz = Z.opa_math.make_norm(np.random.uniform(-1, 1, 3))
    dz = Z.opa_math.make_orth_norm(dz, dy)
    r1 = Z.opa_math.rot_look_at(dz, dy)
    return r1
   
def stereo_XY2xyz(X, Y, r):
    X = X/2/r
    Y = Y/2/r
    
    a = X**2 + Y**2
    return np.array([2*X, 2*Y, -a+1])/(a+1) * r

xl = np.linspace(0, 0, 1)
xl = np.linspace(-0.10, 0.10, 10)
tb = np.stack(np.meshgrid(xl, xl), axis=-1)
X,Y = tb.reshape((-1, 2)).T
vecs = stereo_XY2xyz(X,Y,1).T


In [452]:
alt = 10000.
vx = [0, 0, 1]
flight.set_rotation(tuple(unity_ylookat(vx, dz=np.ones(3)).as_quat()))
pos = earth.position_at_altitude(90., 0., alt, earth.reference_frame)
print(pos, dir * vnorm)
vnorm = 10
for dir in vecs:
    flight.set_rotation(tuple(unity_ylookat(tuple(dir), dz=None).as_quat()))
    fx = flight.simulate_aerodynamic_force_at(earth, pos, tuple(dir*vnorm))
    print(dir, v, fx)
    v =np.abs(np.dot(fx, dir))

(0.0, 610000.0, 0.0) [-0.995 -0.995 9.900]
[-0.100 -0.100 0.990] 117.42457214444876 (11.684160594987524, 11.684159278869629, -116.25737681864729)
[-0.077 -0.100 0.992] 117.425792957154 (9.096618762116933, 11.69565486907959, -116.48728371270282)
[-0.055 -0.100 0.993] 117.42582374624263 (6.502381399985971, 11.704291343688965, -116.65998901543287)
[-0.033 -0.100 0.994] 117.42582525496574 (3.9033562416433867, 11.710054397583008, -116.77526017989103)
[-0.011 -0.100 0.995] 117.4258189412565 (1.3014397448507538, 11.71293830871582, -116.83294011530357)
[0.011 -0.100 0.995] 117.4258172857203 (-1.3014280208365108, 11.712915420532227, -116.83272346330271)
[0.033 -0.100 0.994] 117.42559931460633 (-3.903343626378252, 11.710014343261719, -116.77485810233622)
[0.055 -0.100 0.993] 117.42541467759742 (-6.502347608590043, 11.7042236328125, -116.65930586672965)
[0.077 -0.100 0.992] 117.42513794148596 (-9.096556422063081, 11.695570945739746, -116.4864500612416)
[0.100 -0.100 0.990] 117.42498356975291 (-11

[0.033 0.078 0.996] 117.42661630141156 (-3.9072143418365286, -9.116825103759766, -117.00649093234283)
[0.055 0.078 0.995] 117.42615442986289 (-6.508779996326112, -9.112299919128418, -116.89054888421231)
[0.078 0.078 0.994] 117.42571543452107 (-9.105531381043981, -9.105531692504883, -116.71701812752345)
[0.100 0.077 0.992] 117.4252261216261 (-11.695547583490658, -9.096540451049805, -116.48625879848153)
[-0.100 0.100 0.990] 117.42479027156944 (11.684010267888176, -11.684009552001953, -116.25589085487617)
[-0.077 0.100 0.992] 117.42429192297593 (9.096679592086026, -11.695735931396484, -116.48808383512845)
[-0.055 0.100 0.993] 117.42663025974106 (6.502410667155786, -11.704329490661621, -116.66037579941992)
[-0.033 0.100 0.994] 117.42621493930632 (3.9033669161648348, -11.710095405578613, -116.77567430176163)
[-0.011 0.100 0.995] 117.42623521309483 (1.3014395889304708, -11.712930679321289, -116.83286719730062)
[0.011 0.100 0.995] 117.4257439731352 (-1.301429891857424, -11.712882041931152, -1

In [274]:
ef = earth.reference_frame
ef.

In [162]:
vecs[0], vecs[-1]

(array([-0.100, -0.100, 0.990]), array([0.100, 0.100, 0.990]))

In [36]:
r1= R.from_quat(flight.rotation)
r1.as_matrix()

array([[0.273, 0.050, -0.961],
       [-0.955, 0.132, -0.265],
       [0.114, 0.990, 0.084]])

In [ ]:
for r in cnds:
    r = earth.equatorial_radius
    flight.simulate_aerodynamic_force_at(earth, (r,0,0), flight.direction)

In [18]:
flight.direction

(-0.4712874349752121, 0.08317417173994572, 0.8780490936102943)

In [19]:
r = earth.equatorial_radius
flight.simulate_aerodynamic_force_at(earth, (r,0,0), flight.direction)

(10.374248456866809, -1.275656819343567, -18.72557826952789)

In [ ]:
earth.altitude_at_position(vpos,earth.reference_frame)